# Importaciones

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow gdown fastapi uvicorn nest_asyncio pyngrok --quiet

In [ ]:
import io
import numpy as np
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from threading import Thread
from getpass import getpass
import gdown
import requests
from google.colab import files

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Importar Modelo
url = 'https://drive.google.com/uc?id=1JmfrhsnctvXNG4DI0Ik8QuKA0JG5Ay7X'
output = "model_cnn_cargado.h5"
gdown.download(url, output, quiet=False)
model_cnn = load_model(output)
print("Modelo cargado correctamente.")

# Montaje Servidor

In [ ]:
ALLOWED_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp')

In [ ]:
app = FastAPI(title="API Digitos Lenguaje Signos")
@app.post("/predecir/")
async def predict(file: UploadFile = File(...)):
  try:
        filename = file.filename.lower()

        # Validar extensión
        if not filename.endswith(ALLOWED_EXTENSIONS):
            return JSONResponse(
                status_code=400,
                content={"error": "Formato de imagen no permitido. Solo se aceptan: .jpg, .jpeg, .png, .bmp"}
            )

        # Leer bytes
        img_bytes = await file.read()
        img = Image.open(io.BytesIO(img_bytes))

        # Preprocesamiento de la imagen
        img = img.convert("L")         # Escala de grises
        img = img.resize((100, 100))   # Tamaño de la imagen
        img_array = image.img_to_array(img)
        img_array = img_array / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        pred = model_cnn.predict(img_array)
        numero = int(np.argmax(pred))
        confianza = float(pred[0][numero])

        return {
            "resultado": numero,
            "confianza": confianza
        }

  except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

In [ ]:
# Pedir el token
authtoken = getpass("Introduce tu authtoken de ngrok: ")
# Configurar ngrok
!ngrok authtoken {authtoken}

In [ ]:
ngrok.kill()  # cerrar túneles previos

In [ ]:
# Elegir un puerto libre (5000 y 8888 van bien)
PORT = 8888

# Ejecutar el servidor (Uvicorn)
nest_asyncio.apply()

def run():
    uvicorn.run(app, host="0.0.0.0", port=PORT)

# Iniciar FastAPI en un hilo
Thread(target=run).start()

# Abrir túnel público con ngrok
tunnel = ngrok.connect(PORT)
public_url = tunnel.public_url
print("Tu URL pública es:", public_url)

# Solicitud de predicción al Servidor

In [ ]:
url_publica= f"{public_url}/predecir/"

In [ ]:
uploaded = files.upload()

In [ ]:
nombre_archivo = "IMG_1275_num5.JPG" # Cambiar por el nombre de la imagen subida
files = {"file": open(nombre_archivo, "rb")}

In [ ]:
img = Image.open(nombre_archivo)
plt.figure(figsize=(2,2))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Hacer la petición POST al endpoint FastAPI
response = requests.post(url_publica, files=files)
print(response.status_code, response.text)